# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
import os, json
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display
from scraper import *



load_dotenv(override=True)


gemini_model={
	'base_url': "https://generativelanguage.googleapis.com/v1beta/openai/",
	'models': ['gemini-2.5-flash-lite'],
	'api_key': os.getenv('GOOGLE_API_KEY'),
}
openai_model={
	'base_url': "https://api.openai.com/v1",
	'models': ['gpt-4.1-nano', 'gpt-4.1-mini', 'gpt-5-nano', 'gpt-5-nano',],
	'api_key': os.getenv('OPENAI_API_KEY'),	
}
ollama_model={
	'base_url': "http://localhost:11434/v1",
	'models': ['llama3.2'],
	'api_key': None,
}
MODEL = {
	'gemini':gemini_model,
	'openai':openai_model,
	'ollama':ollama_model,
}

urls=[
	"https://www.metacritic.com/music/lux/rosalia/critic-reviews",
]


In [ ]:

model= 'openai'
idx_models = 1
toggle_stream=False
format_type={"type": "json_object"}  ### None: allowed, {}: not allowed


system_prompt=f"""
you are provided with a lot of links
you have to filter some of them whether or not the given urls are relevant to the critics' review for an albums.
you are allowed to answer in json format like below

{{
	"critics_reviews": [
		{{"critic": "Rolling Stone UK", "url": "https://www.rolling.stone.com/19392"}},
		{{"critic": "Pitchfork", "url": "https://www.pitchfork.review.com/album/2026/3531"}},
	]
}}

"""

user_prompt=f"""
here are the links for a specific album, url: {urls[0]}.
you have to filter these links to only include a relevent links by the critics 

"""

user_prompt += "\n".join(fetch_website_links(urls[0]))



openai = OpenAI(base_url=MODEL[model]['base_url'], api_key=MODEL[model]['api_key'],)



prompt_messages = [
	{'role':'system', 'content':system_prompt},
	{'role':'user',   'content':user_prompt},
]

response_obj = openai.chat.completions.create(
	model = MODEL[model]['models'][idx_models],
	messages = prompt_messages,
	stream = toggle_stream,
	response_format = format_type,
)


if toggle_stream:
	response = ""
	display_handle = display(Markdown(response), display_id=True)
	for chunk in response_obj:
		response += chunk.choices[0].message.content or ''
		update_display(Markdown(response), display_id= display_handle.display_id)
elif not format_type:
	result = response_obj.choices[0].message.content
	display(Markdown(result))	
else:
	result = response_obj.choices[0].message.content
	print(result)


In [ ]:
result_json = json.loads(result)

In [ ]:
model= 'gemini'
idx_models = 0
toggle_stream=True
format_type=None  ### None: allowed, {}: not allowed


system_prompt=f'''
based on what user give, you have to write an overall summary of the critics' review of an album.
you should also append the summary of each critic's review.
'''

user_prompt=f'''
here are the reviews by several critics for an album

'''

for entry in result_json["critics_reviews"]:
	user_prompt += f"##{entry['critic']}\n"
	user_prompt += f"{fetch_website_contents(entry['url'])}\n\n"



openai = OpenAI(base_url=MODEL[model]['base_url'], api_key=MODEL[model]['api_key'],)



prompt_messages = [
	{'role':'system', 'content':system_prompt},
	{'role':'user',   'content':user_prompt},
]

response_obj = openai.chat.completions.create(
	model = MODEL[model]['models'][idx_models],
	messages = prompt_messages,
	stream = toggle_stream,
	response_format = format_type,
)


if toggle_stream:
	response = ""
	display_handle = display(Markdown(response), display_id=True)
	for chunk in response_obj:
		response += chunk.choices[0].delta.content or ''
		update_display(Markdown(response), display_id= display_handle.display_id)
elif not format_type:
	result = response_obj.choices[0].message.content
	display(Markdown(result))	
else:
	result = response_obj.choices[0].message.content
	print(result)

